<a href="https://colab.research.google.com/github/mrrahulkrr/Speaker-Diarization-assignment/blob/main/Speaker_diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: Speaker Diarization Notebook




below are some requirement that you need to do

Notes on usage:

- Make sure to [change runtime to GPU](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- The transcript will be saved in Files, which you can find in the menu on the left.
- Change the number of speakers below if different from two.
- Pick a bigger model if you want more accuracy and a smaller model if you want the program to run faster ([more info](https://github.com/openai/whisper#available-models-and-languages)).
- If you know the language being spoken is English, then change language to 'English' as this improves performance.


Overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.

I hope my assignment covers all expected given mechanisms, Let me know if I can make it better!


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* "From Here any user can upload their own audio files from device as input"

In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving assignment sample audio (online-audio-converter.com).mp3 to assignment sample audio (online-audio-converter.com).mp3


* Now select the speaker language and model you want to add::

In [ ]:
num_speakers = 2

language = 'English'

model_size = 'large'


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


* This method is executed just to convert to encode to UTF 8

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

* Importing the required dependencies like whisper, pyannote, numpy, speechbrain etc

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


(…)a-voxceleb/resolve/main/hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

(…)eleb/resolve/main/mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

(…)-voxceleb/resolve/main/label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

"This function verifies the file format to ensure it is in the WAV format."

In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

* Loading the model

In [ ]:
model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 2.87G/2.87G [00:28<00:00, 109MiB/s]


* "Creating segmented portions of the audio file."

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

* This helps to get the frames rate and duration

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

*  pyannote built in function

In [ ]:
audio = Audio()

In [ ]:
def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)

    # Reshape the waveform tensor to match the expected input shape
    waveform = waveform.unsqueeze(0)

    return embedding_model(waveform)


* This helps to split the audios into wavform

In [ ]:
def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(path, clip)

    # Convert waveform to single channel
    waveform = waveform.mean(dim=0, keepdim=True)

    return embedding_model(waveform.unsqueeze(0))


Here we create the embeddings

In [ ]:

embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)


embeddings = np.nan_to_num(embeddings)

* Here we are using agglomerative clustering to cluster same channels as 1 speaker

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w",encoding="UTF-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:
print(open('transcript.txt','r',encoding="UTF-8").read())


SPEAKER 1 0:00:00
Contextual understanding utilizes language patterns for speaker separation, enhancing accuracy in context-rich conversations. 
SPEAKER 2 0:00:08
Language independence less reliant on specific accents or languages, making it adaptable to diverse linguistic environments. 
SPEAKER 1 0:00:20
Semantic insights captures speaker intent and topic shifts, valuable for applications requiring deeper conversation analysis. 


#↑ This is how you can get the Speaker Diarization ↑